# Test 4: Flowchart Detection with Layout Parser

**Goal:** Detect flowcharts/diagrams and describe them using Vision LLM

This notebook tests the flowchart/diagram detection and description capabilities using **Layout Parser** and **Vision LLM**.

## What This Test Does:
- ✅ Uses Layout Parser to detect images in PDFs
- ✅ Filters for likely diagrams/flowcharts (vs photos)
- ✅ Extracts image regions from PDF pages
- ✅ Uses Vision LLM (GPT-4 Vision or Claude) to generate descriptions
- ✅ Saves descriptions for taxonomy matching

**Why This Matters:**
- Makes visual content searchable
- Creates accessible text versions of diagrams
- Essential for comprehensive textbook processing
- Layout Parser provides accurate image boundary detection


## Step 1: Install Dependencies

Run this cell to install required packages for Google Colab.


In [ ]:
%pip install -q google-cloud-documentai python-dotenv openai anthropic pdf2image Pillow
print("✅ All dependencies installed!")


## Step 2: Upload Credentials

Upload your Google Cloud service account JSON file.


In [ ]:
from google.colab import files
import json
import os

print("📤 Please upload your Google Cloud credentials JSON file...")
uploaded = files.upload()

creds_filename = list(uploaded.keys())[0]
credentials_content = json.loads(uploaded[creds_filename].decode('utf-8'))

with open('docai-credentials.json', 'w') as f:
    json.dump(credentials_content, f)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'docai-credentials.json'
print(f"✅ Credentials saved: {creds_filename}")


## Step 3: Configure Environment

Set your Google Cloud project ID, Layout Parser processor ID, and Vision LLM API key.


In [ ]:
# Configuration - UPDATE THESE VALUES
DOCAI_PROJECT_ID = "your-project-id-here"
DOCAI_PROCESSOR_ID = "your-layout-parser-processor-id"
DOCAI_LOCATION = "us"

# LLM Configuration for flowchart description
OPENAI_API_KEY = "sk-your-openai-key-here"  # Or use ANTHROPIC_API_KEY
LLM_PROVIDER = "openai"  # or "anthropic"

os.environ['DOCAI_PROJECT_ID'] = DOCAI_PROJECT_ID
os.environ['DOCAI_PROCESSOR_ID'] = DOCAI_PROCESSOR_ID
os.environ['DOCAI_LOCATION'] = DOCAI_LOCATION
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LLM_PROVIDER'] = LLM_PROVIDER

print(f"✅ Configuration set:")


## Step 4: Clone Repository and Load Utils

Clone the repository to access utility functions.


In [ ]:
!git clone https://github.com/abhii-01/python-automation.git
%cd python-automation

import sys
from pathlib import Path
sys.path.append(str(Path.cwd()))

from utils.docai_client import get_client_from_env
from utils.vision_llm import describe_image_with_llm, extract_image_from_pdf, is_likely_diagram

print("✅ Repository cloned and utilities loaded")


## Step 5: Verify Setup

Test connection to Google Document AI.


In [ ]:
print("🔍 Verifying Document AI setup...\n")

try:
    client = get_client_from_env()
    client.verify_setup()
    print("\n✅ Setup verified! Ready to process documents.")
except Exception as e:
    print(f"\n❌ Setup verification failed: {e}")


## Step 6: Upload PDF for Testing

Upload your PDF file with diagrams/flowcharts to process.


In [ ]:
print("📤 Please upload your PDF file (should contain diagrams/flowcharts)...")
uploaded_pdfs = files.upload()

pdf_filename = list(uploaded_pdfs.keys())[0]
pdf_path = pdf_filename

print(f"✅ PDF uploaded: {pdf_filename}")
print(f"   Size: {len(uploaded_pdfs[pdf_filename]) / 1024:.1f} KB")


## Step 7: Process Document with Layout Parser

Process the PDF to detect images.


In [ ]:
print(f"{'='*60}")
print("TEST 4: FLOWCHART DETECTION")
print(f"{'='*60}\n")

print(f"📄 Processing PDF with Layout Parser: {pdf_path}")
document = client.process_document(pdf_path)

print(f"✅ Document processed!")
print(f"   Total pages: {len(document.pages)}")


## Step 8: Define Helper Functions


In [ ]:
def get_bounding_box(bounding_poly):
    """Extract normalized bounding box from polygon"""
    if not bounding_poly or not hasattr(bounding_poly, 'normalized_vertices'):
        return {"x_min": 0, "y_min": 0, "x_max": 0, "y_max": 0}
    
    vertices = bounding_poly.normalized_vertices
    if not vertices:
        return {"x_min": 0, "y_min": 0, "x_max": 0, "y_max": 0}
    
    x_coords = [v.x for v in vertices]
    y_coords = [v.y for v in vertices]
    
    return {
        "x_min": min(x_coords),
        "y_min": min(y_coords),
        "x_max": max(x_coords),
        "y_max": max(y_coords)
    }

def get_page_text(page, full_text):
    """Extract text from a page"""
    text_parts = []
    for paragraph in page.paragraphs:
        if paragraph.layout.text_anchor:
            for segment in paragraph.layout.text_anchor.text_segments:
                text = full_text[segment.start_index:segment.end_index]
                text_parts.append(text)
    return " ".join(text_parts)

print("✅ Helper functions defined")


## Step 9: Detect and Process Flowcharts/Diagrams

Extract images and generate descriptions using Vision LLM.


In [ ]:
print("\n🔍 Detecting diagrams and flowcharts...\n")

diagram_results = []
diagram_count = 0

for page_num, page in enumerate(document.pages, 1):
    # Check for images on this page
    if not hasattr(page, 'image') or not page.image:
        continue
    
    print(f"  Page {page_num}: Found {len(page.image)} image(s)")
    
    for img_idx, image in enumerate(page.image):
        # Get bounding box
        bbox = get_bounding_box(image.layout.bounding_poly)
        
        # Get page text for context
        page_text = get_page_text(page, document.text)
        
        # Check if likely a diagram
        if not is_likely_diagram(bbox, page_text):
            print(f"    Image {img_idx + 1}: Skipped (likely photo/decoration)")
            continue
        
        diagram_count += 1
        print(f"    Image {img_idx + 1}: Detected as diagram")
        
        # Extract image region from PDF
        print(f"      Extracting image region...")
        try:
            image_bytes = extract_image_from_pdf(pdf_path, page_num - 1, bbox)
            print(f"      ✅ Extracted ({len(image_bytes)} bytes)")
        except Exception as e:
            print(f"      ⚠️  Extraction failed: {e}")
            continue
        
        # Describe with Vision LLM
        print(f"      🤖 Generating description with Vision LLM...")
        try:
            description = describe_image_with_llm(
                image_bytes,
                image_type="flowchart"
            )
            print(f"      ✅ Description generated ({len(description)} chars)")
        except Exception as e:
            print(f"      ⚠️  LLM description failed: {e}")
            description = f"[Description generation failed: {e}]"
        
        # Store result
        result = {
            "diagram_id": f"diagram_{diagram_count}",
            "page": page_num,
            "image_index": img_idx,
            "bbox": bbox,
            "area_percentage": (bbox['x_max'] - bbox['x_min']) * (bbox['y_max'] - bbox['y_min']) * 100,
            "description": description
        }
        
        diagram_results.append(result)

print(f"\n✅ Processed {diagram_count} diagrams")


## Step 10: View Results

Display detected diagrams and their descriptions.


In [ ]:
print(f"{'='*60}")
print("✅ FLOWCHART DETECTION COMPLETE")
print(f"{'='*60}")
print(f"📊 Summary:")
print(f"  Total diagrams found: {diagram_count}")
print(f"  Successfully described: {len(diagram_results)}")

if diagram_results:
    print(f"\n📝 Example description (Diagram 1):")
    print("-" * 60)
    example_desc = diagram_results[0]['description']
    print(example_desc[:400])
    if len(example_desc) > 400:
        print("...")
    print("-" * 60)
else:
    print("\n⚠️  No diagrams found in this document")


## Step 11: Save Results to JSON


In [ ]:
from pathlib import Path

results = {
    "pdf_file": Path(pdf_path).name,
    "total_diagrams": diagram_count,
    "diagrams": diagram_results
}

output_path = "test4_flowcharts.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_path}")


## Step 12: Download Results

Download the JSON results file to your computer.


In [ ]:
files.download(output_path)
print(f"✅ Test 4 complete! Results downloaded.")
